# WAV_motorcycle_MySQL_적재

## 0. Spark Session 생성

In [1]:
from pyspark.sql import SparkSession

# MySQL JDBC 드라이버 경로 (압축 푼 드라이버 JAR 파일 경로)
mysql_driver_path = "/home/ubuntu/mysql-connector-j-9.2.0/mysql-connector-j-9.2.0.jar"


# SparkSession 생성
spark = SparkSession.builder \
    .appName("WAV_motor") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.jars", mysql_driver_path) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .getOrCreate()

## 1. wav_motor_horn_data

In [8]:
import pyspark
from pyspark.sql import SparkSession
import io
from scipy.io import wavfile
import librosa
import numpy as np
import os
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType

# HDFS에서 모든 WAV 파일 읽기
hdfs_dir = "hdfs://localhost:9000/shared_data/raw_data/2.Motorcycle/4.horn_of_motorcycle"
binary_df = spark.read.format("binaryFile").load(hdfs_dir)

# 🔹 UDF (User Defined Function) 정의: WAV → MFCC 변환
def extract_mfcc(binary_data):
    try:
        audio_bytes = io.BytesIO(binary_data)  # 바이너리 데이터를 메모리 파일로 변환
        sr, audio = wavfile.read(audio_bytes)  # scipy로 샘플링 레이트 확인
        audio_librosa, sr_librosa = librosa.load(audio_bytes, sr=None)  # librosa로 리샘플링
        mfcc = librosa.feature.mfcc(y=audio_librosa, sr=sr_librosa, n_mfcc=13)  # MFCC 추출
        mfcc_mean = np.mean(mfcc, axis=1).astype(float)  # 평균 계산
        return mfcc_mean.tolist()  # 리스트로 반환
    except Exception as e:
        return None  # 에러 발생 시 None 반환

# UDF 등록
mfcc_udf = udf(extract_mfcc, ArrayType(FloatType()))

# 🔹 파일 이름 추출 UDF 정의
def extract_filename(path):
    return os.path.basename(path)

filename_udf = udf(extract_filename, StringType())

# 🔹 변환 적용
df_mfcc = binary_df \
    .withColumn("fileName", filename_udf(binary_df["path"])) \
    .withColumn("mfcc_features", mfcc_udf(binary_df["content"]))

# 🔹 배열 데이터를 개별 컬럼으로 변환
mfcc_columns = [f"mfcc_{i+1}" for i in range(13)]
for i in range(13):
    df_mfcc = df_mfcc.withColumn(mfcc_columns[i], df_mfcc["mfcc_features"][i])

# 🔹 불필요한 컬럼 정리
df_mfcc = df_mfcc.select(["fileName"] + mfcc_columns)

# 🔹 결과 저장 (HDFS)
# output_path = "hdfs://localhost:9000/shared_data/mfcc_features/"
# df_mfcc.write.csv(output_path, header=True, mode="overwrite")

# print(f"✅ MFCC 데이터가 HDFS에 저장됨: {output_path}")

In [3]:
df_mfcc.count()

4560

In [4]:
df_mfcc.createOrReplaceTempView("wav_motor_horn")

# SQL 쿼리 실행하여 데이터 추출
df_mfcc = spark.sql("""
    SELECT *
    FROM wav_motor_horn
""")

# MySQL연결
mysql_url = "jdbc:mysql://15.168.145.74:3306/my_db?useUnicode=true&characterEncoding=UTF-8"
mysql_properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# MySQL로 DataFrame 적재 (쿼리 결과가 None이 아닌 경우에만)
if df_mfcc is not None:
    df_mfcc.write.jdbc(url=mysql_url, table="wav_motorcycle_horn_data", mode="overwrite", properties=mysql_properties)
    print("데이터가 MySQL로 성공적으로 적재되었습니다!")
else:
    print("쿼리 결과가 없습니다. 데이터 추출이 실패했습니다.")


데이터가 MySQL로 성공적으로 적재되었습니다!


## 2. wav_motor_driving_data

In [9]:
import pyspark
from pyspark.sql import SparkSession
import io
from scipy.io import wavfile
import librosa
import numpy as np
import os
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType

# HDFS에서 모든 WAV 파일 읽기
hdfs_dir = "hdfs://localhost:9000/shared_data/raw_data/2.Motorcycle/5.driving_sound_of_motorcycle"
binary_df = spark.read.format("binaryFile").load(hdfs_dir)

# 🔹 UDF (User Defined Function) 정의: WAV → MFCC 변환
def extract_mfcc(binary_data):
    try:
        audio_bytes = io.BytesIO(binary_data)  # 바이너리 데이터를 메모리 파일로 변환
        sr, audio = wavfile.read(audio_bytes)  # scipy로 샘플링 레이트 확인
        audio_librosa, sr_librosa = librosa.load(audio_bytes, sr=None)  # librosa로 리샘플링
        mfcc = librosa.feature.mfcc(y=audio_librosa, sr=sr_librosa, n_mfcc=13)  # MFCC 추출
        mfcc_mean = np.mean(mfcc, axis=1).astype(float)  # 평균 계산
        return mfcc_mean.tolist()  # 리스트로 반환
    except Exception as e:
        return None  # 에러 발생 시 None 반환

# UDF 등록
mfcc_udf = udf(extract_mfcc, ArrayType(FloatType()))

# 🔹 파일 이름 추출 UDF 정의
def extract_filename(path):
    return os.path.basename(path)

filename_udf = udf(extract_filename, StringType())

# 🔹 변환 적용
df_mfcc = binary_df \
    .withColumn("fileName", filename_udf(binary_df["path"])) \
    .withColumn("mfcc_features", mfcc_udf(binary_df["content"]))

# 🔹 배열 데이터를 개별 컬럼으로 변환
mfcc_columns = [f"mfcc_{i+1}" for i in range(13)]
for i in range(13):
    df_mfcc = df_mfcc.withColumn(mfcc_columns[i], df_mfcc["mfcc_features"][i])

# 🔹 불필요한 컬럼 정리
df_mfcc = df_mfcc.select(["fileName"] + mfcc_columns)

# 🔹 결과 저장 (HDFS)
# output_path = "hdfs://localhost:9000/shared_data/mfcc_features/"
# df_mfcc.write.csv(output_path, header=True, mode="overwrite")

# print(f"✅ MFCC 데이터가 HDFS에 저장됨: {output_path}")

In [10]:
df_mfcc.count()

4735

In [11]:
df_mfcc.createOrReplaceTempView("wav_motor_driving")

# SQL 쿼리 실행하여 데이터 추출
df_mfcc = spark.sql("""
    SELECT *
    FROM wav_motor_driving
""")

# MySQL연결
mysql_url = "jdbc:mysql://15.168.145.74:3306/my_db?useUnicode=true&characterEncoding=UTF-8"
mysql_properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# MySQL로 DataFrame 적재 (쿼리 결과가 None이 아닌 경우에만)
if df_mfcc is not None:
    df_mfcc.write.jdbc(url=mysql_url, table="wav_motorcycle_driving_data", mode="overwrite", properties=mysql_properties)
    print("데이터가 MySQL로 성공적으로 적재되었습니다!")
else:
    print("쿼리 결과가 없습니다. 데이터 추출이 실패했습니다.")


데이터가 MySQL로 성공적으로 적재되었습니다!


In [12]:
spark.stop()